#### Before start:
This analysis presumes some basic knowledges in the PC game League of Legends, so before you proceed, here is a brief introduction to some of the concepts in the game:

#### **Introduction to League of Legends**

League of Legends is a Multiplayer Online Battle Arena (MOBA) game, where two teams of five players each compete to destroy the opposing team's base to claim victory. Each player controls a unique champion, and teamwork is crucial for success.

**Key Concepts:**
* Champion: each user controls a unique champion in the game, and often times champions in a team will need to collaborate well to win.
* Objectives: monsters which both teams will try to slay, as they will bring massive benefits in order to win a game.
    1. Baron Nashor: The most powerful monster, granting significant buffs.
    2. Drakes (Dragons): Elemental creatures that offer different bonuses based on their type.
* Items: Equipment purchased using in-game gold to enhance a champion’s abilities.
* Vision & Wards: Vision allows teams to see enemies on the map. This is achieved by placing wards, which provide visibility in specific areas.

    
***
Source: https://leagueoflegends.fandom.com/wiki/League_of_Legends_Wiki, https://euw.leagueoflegends.com/en-gb/how-to-play/
***

## **Statistically Winning League of Legends**

### **Project Summary**
How can you consistently win in League of Legends if you're not a mechanically gifted player like *Rookie*, *Uzi*, or *Faker*? One practical approach is to analyze and replicate the strategies of successful players.

This project examines 100,800 high-ranked games from the Korean (KR) server to answer key questions that have long puzzled players:

* Who are the best buddies in champions?
* What are the most effective item builds for each champion?
* Which is more valuable: securing the Dragon Soul (four dragons) or Baron Nashor?
* Does vision control significantly impact win rates?

### **Project Scope**
The analysis will focus on three core aspects of the game:
1. Champion compositions – Identifying optimal champion pairings.
2. Item builds – Determining the most effective items for each champion.
3. Team objectives & vision control – Assessing the impact of objectives and vision on winning.

### **Project Workflow**
1. Scope the Project & Gather Data
2. Explore & Assess the Data
3. Define the Data Model
4. Run ETL to Model the Data
5. Complete Project Write Up

In [1]:
%%capture

!pip install kaggle
!pip install --upgrade pip
!pip install psycopg2-binary

In [2]:
import os
import json                         # data processing, json file I/O
import random                       # data quality testing
import kaggle
import psycopg2                     # establish aws Redshift connection
import sqlalchemy                   # copy pd dataframe to Redshift 
import numpy as np                  # linear algebra
import pandas as pd                 # data processing, CSV file I/O (e.g. pd.read_csv)
from sqlalchemy.types import *      # load staging_tables

In [3]:
datasets = {
            "lol_ranked_games": "gyejr95/league-of-legendslol-ranked-games-2020-ver1",
            "lol_data_dragon": "peter0228/league-of-legendslol-data-dragon-en-us10151"
}

dataframes = {}
for name, dataset in datasets.items():
    kaggle.api.dataset_download_files(dataset, path=".", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/gyejr95/league-of-legendslol-ranked-games-2020-ver1
Dataset URL: https://www.kaggle.com/datasets/peter0228/league-of-legendslol-data-dragon-en-us10151


## Step 1: Scope the Project and Gather Data

#### **Scope**
The end-result is prepared for a backend of game analytics, to give suggestions on winning strategies. The two sources of data are: [League of Legends(LOL) - Ranked Games 2020](https://www.kaggle.com/gyejr95/league-of-legendslol-ranked-games-2020-ver1?select=match_data_version1.csv), a Kaggle dataset; [Data Dragon of Patch 10.15.1](https://ddragon.leagueoflegends.com/cdn/dragontail-10.15.1.tgz), LOL game data and assets from Riot (LOL's monther company).

The end solution will be a database in aws RDS Postgres, containing the data to full solutions of each question.

Some Python analytic libraries are used in imports.

#### **Describe and Gather Data**
The Kaggle dataset contains 10,800 game data in the KR server, the Data Dragon (meta_champs & meta_items) contains all the in-game data up to patch 10.15.1, which is the latest version of game by Aug. 2, 2020.


There are three match data files:

* `match_df` – Contains participant details for both winning and losing teams.
* `winner_df` – Includes in-game stats and objective data for the winning team.
* `loser_df` – Contains the same stats and objective data for the losing team.

In [4]:
match_df = pd.read_pickle('match_data_version1.pickle')
winner_df =  pd.read_pickle('match_winner_data_version1.pickle')
loser_df = pd.read_pickle('match_loser_data_version1.pickle')

In [5]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108829 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   gameCreation           108829 non-null  float64
 1   gameDuration           108829 non-null  float64
 2   gameId                 108829 non-null  float64
 3   gameMode               108829 non-null  object 
 4   gameType               108829 non-null  object 
 5   gameVersion            108829 non-null  object 
 6   mapId                  108829 non-null  float64
 7   participantIdentities  108829 non-null  object 
 8   participants           108829 non-null  object 
 9   platformId             108829 non-null  object 
 10  queueId                108829 non-null  float64
 11  seasonId               108829 non-null  float64
 12  status.message         0 non-null       object 
 13  status.status_code     0 non-null       float64
dtypes: float64(7), object(7)
memory usage: 12

In [6]:
match_df['gameVersion'].iloc[0]

'10.6.314.4405'

In [7]:
winner_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108829 entries, 0 to 108828
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   teamId                108829 non-null  int64  
 1   win                   108829 non-null  object 
 2   firstBlood            108829 non-null  bool   
 3   firstTower            108829 non-null  bool   
 4   firstInhibitor        108829 non-null  bool   
 5   firstBaron            108829 non-null  bool   
 6   firstDragon           108829 non-null  bool   
 7   firstRiftHerald       108829 non-null  bool   
 8   towerKills            108829 non-null  int64  
 9   inhibitorKills        108829 non-null  int64  
 10  baronKills            108829 non-null  int64  
 11  dragonKills           108829 non-null  int64  
 12  vilemawKills          108829 non-null  int64  
 13  riftHeraldKills       108829 non-null  int64  
 14  dominionVictoryScore  108829 non-null  int64  
 15  

In [8]:
loser_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108829 entries, 0 to 108828
Data columns (total 17 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   teamId                108829 non-null  int64  
 1   win                   108817 non-null  object 
 2   firstBlood            108829 non-null  bool   
 3   firstTower            108829 non-null  bool   
 4   firstInhibitor        108829 non-null  bool   
 5   firstBaron            108829 non-null  bool   
 6   firstDragon           108829 non-null  bool   
 7   firstRiftHerald       108829 non-null  bool   
 8   towerKills            108829 non-null  int64  
 9   inhibitorKills        108829 non-null  int64  
 10  baronKills            108829 non-null  int64  
 11  dragonKills           108829 non-null  int64  
 12  vilemawKills          108829 non-null  int64  
 13  riftHeraldKills       108829 non-null  int64  
 14  dominionVictoryScore  108829 non-null  int64  
 15  

Read the `meta_*.json` files, which store information about all in-game entities in League of Legends, including champions, items, and more.

In [9]:
meta_champs = pd.read_json('en_US-10.15.1/meta_champion.json')

In [10]:
meta_champs.iloc[0]['data']

{'version': '10.15.1',
 'id': 'Aatrox',
 'key': '266',
 'name': 'Aatrox',
 'title': 'the Darkin Blade',
 'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
 'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
 'image': {'full': 'Aatrox.png',
  'sprite': 'champion0.png',
  'group': 'champion',
  'x': 0,
  'y': 0,
  'w': 48,
  'h': 48},
 'tags': ['Fighter', 'Tank'],
 'partype': 'Blood Well',
 'stats': {'hp': 580,
  'hpperlevel': 90,
  'mp': 0,
  'mpperlevel': 0,
  'movespeed': 345,
  'armor': 38,
  'armorperlevel': 3.25,
  'spellblock': 32.1,
  'spellblockperlevel': 1.25,
  'attackrange': 175,
  'hpregen': 3,
  'hpregenperlevel': 1,
  'mpregen': 0,
  'mpregenperlevel': 0,
  'crit': 0,
  'critperlevel': 0,
  'attackdamage': 60,
  'attackdamageperlevel': 5,
  'at

From `meta_champs`, we only need the key-name mapping for each champion, as it is referenced in all match tables.

Similarly, we need the 'key-name' mapping for each item from `meta_items`, as it is used in the match data.

In [11]:
with open('en_US-10.15.1/meta_item.json') as f:
    data = json.load(f)

meta_items = pd.read_json(json.dumps(data['data']), orient='index')

In [12]:
meta_items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 1001 to 4403
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              264 non-null    object 
 1   description       264 non-null    object 
 2   colloq            264 non-null    object 
 3   plaintext         264 non-null    object 
 4   into              84 non-null     object 
 5   image             264 non-null    object 
 6   gold              264 non-null    object 
 7   tags              264 non-null    object 
 8   maps              264 non-null    object 
 9   stats             264 non-null    object 
 10  from              165 non-null    object 
 11  depth             165 non-null    float64
 12  effect            212 non-null    object 
 13  hideFromAll       26 non-null     float64
 14  stacks            16 non-null     float64
 15  consumed          19 non-null     float64
 16  inStore           34 non-null     float6

In [13]:
meta_items.iloc[0]

name                                                   Boots of Speed
description         <groupLimit>Limited to 1 pair of boots.</group...
colloq                                                              ;
plaintext                           Slightly increases Movement Speed
into                 [3006, 3047, 3020, 3158, 3111, 3117, 3009, 4001]
image               {'full': '1001.png', 'sprite': 'item0.png', 'g...
gold                {'base': 300, 'purchasable': True, 'total': 30...
tags                                                          [Boots]
maps                {'11': True, '12': True, '21': True, '22': False}
stats                                    {'FlatMovementSpeedMod': 25}
from                                                              NaN
depth                                                             NaN
effect                                                            NaN
hideFromAll                                                       NaN
stacks              

## Step 2: Explore and Assess the Data

#### **Explore the Data**

Identified issues:
1. **NaN** values in the `win` column of `loser_df`.
2. `gameMode` in `match_df` - We only need matches with the **classic** mode, as these represent ranked games.
3. **Remake games** – Matches that ended early and should be excluded.

#### Cleaning Steps
Since `gameId` is the foreign key in `match_df`, we will use it to remove affected records across all three dataframes.

#### ***Issue 1***

In [14]:
loser_df['win'].isna().sum()

12

There are 12 **NaN** values in the `win` field of `loser_df`, which should presumably be **Fail** instead.

To verify this, we will:
1. Find their corresponding opponents in `winner_df` using `gameId` (the foreign key in both datasets) to check if these are truly missing values.
2. Cross-check the full match details in `match_df` for further validation.

In [15]:
loser_df[loser_df['win'].isna() == True]

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId
29877,200,NaN,False,True,False,False,False,False,1,0,0,0,0,0,0,[],4.235638e+09
44076,200,NaN,False,True,False,False,False,False,1,0,0,0,0,0,0,[],4.189115e+09
68215,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.092070e+09
77450,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.092091e+09
83300,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.131950e+09
85130,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.060127e+09
86529,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.092111e+09
88618,200,NaN,False,True,False,False,False,False,1,0,0,0,0,0,0,[],4.176526e+09
92195,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.092124e+09
93574,200,NaN,False,True,False,False,False,False,5,0,0,0,0,0,0,[],4.048611e+09


In [16]:
missingVals = loser_df[loser_df['win'].isna() == True]['gameId'].tolist()
winner_df[winner_df['gameId'].isin(missingVals)]

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore,bans,gameId
29877,100,Win,True,False,True,False,False,False,7,1,0,0,0,0,0,[],4.235638e+09
44076,100,Win,True,False,True,False,False,False,7,1,0,0,0,0,0,[],4.189115e+09
68215,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.092070e+09
77450,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.092091e+09
83300,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.131950e+09
85130,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.060127e+09
86529,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.092111e+09
88618,100,Win,True,False,True,False,False,False,7,1,0,0,0,0,0,[],4.176526e+09
92195,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.092124e+09
93574,100,Win,True,False,True,False,False,False,11,1,0,0,0,0,0,[],4.048611e+09


In [17]:
match_df[match_df['gameId'].isin(missingVals)]

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status.message,status.status_code
29877,1.584799e+12,219.0,4.235638e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
44076,1.583121e+12,237.0,4.189115e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.4.308.9400,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.579283e+12,157.0,4.092070e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.1.303.9385,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.579283e+12,144.0,4.092091e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.1.303.9385,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.580859e+12,296.0,4.131950e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.3.307.1028,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.577894e+12,242.0,4.060127e+09,TUTORIAL_MODULE_1,MATCHED_GAME,9.24.300.6382,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.579283e+12,144.0,4.092111e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.1.303.9385,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.582644e+12,526.0,4.176526e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.4.308.9400,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.579285e+12,132.0,4.092124e+09,TUTORIAL_MODULE_1,MATCHED_GAME,10.1.303.9385,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN
0,1.577416e+12,241.0,4.048611e+09,TUTORIAL_MODULE_1,MATCHED_GAME,9.24.300.6382,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,2000.0,13.0,NaN,NaN


All 12 games belong to **TUTORIAL_MODULE_1**, which is a tutorial mode rather than a standard match in League of Legends.

To compare, we will examine a random subset of rows from both `loser_df` and `match_df` to ensure these tutorial games differ from regular matches.

In [18]:
randomlist = []

for i in range(10):
    n = random.randint(0, 108828)
    randomlist.append(n)

print(randomlist)

[12164, 45101, 70948, 102764, 55939, 33845, 63508, 18963, 55083, 89180]


In [19]:
match_df[match_df['gameId'].isin(loser_df.iloc[randomlist]['gameId']).tolist()]

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status.message,status.status_code
12164,1.585406e+12,1444.0,4.255280e+09,CLASSIC,MATCHED_GAME,10.6.314.4405,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,430.0,13.0,NaN,NaN
18963,1.583477e+12,917.0,4.198331e+09,CLASSIC,MATCHED_GAME,10.5.311.166,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
33845,1.581902e+12,1297.0,4.157697e+09,CLASSIC,MATCHED_GAME,10.3.307.7898,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
45101,1.584610e+12,1543.0,4.228871e+09,CLASSIC,MATCHED_GAME,10.6.313.2784,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
55083,1.581515e+12,2200.0,4.147999e+09,CLASSIC,MATCHED_GAME,10.3.307.7898,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
55939,1.585078e+12,1275.0,4.244733e+09,CLASSIC,MATCHED_GAME,10.6.313.8894,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,430.0,13.0,NaN,NaN
63508,1.584271e+12,1829.0,4.219983e+09,CLASSIC,MATCHED_GAME,10.5.312.392,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
0,1.582449e+12,1861.0,4.171244e+09,CLASSIC,MATCHED_GAME,10.4.308.9400,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN
0,1.576635e+12,644.0,4.032348e+09,KINGPORO,MATCHED_GAME,9.24.300.6382,12.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,920.0,13.0,NaN,NaN
0,1.584274e+12,2233.0,4.220077e+09,CLASSIC,MATCHED_GAME,10.5.312.392,11.0,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420.0,13.0,NaN,NaN


A key observation is the dramatic difference in `gameDuration` between the random set and the win == NaN set. Unfortunately, there is no official documentation on `gameDuration` in the Riot API.

A reasonable assumption is that `gameDuration` is measured in **seconds**, as this aligns with the values observed in the randomly selected set.

In [20]:
avg_gameDuration = np.average(match_df[match_df['gameId'].isin(loser_df.iloc[randomlist]['gameId'])]['gameDuration'].values)

print('if unit in sec: {:.2f} sec'.format(avg_gameDuration))
print('if unit in mins: {:.2f} mins'.format(avg_gameDuration/60))
print('if unit in hours: {:.2f} hours'.format(avg_gameDuration/3600))

if unit in sec: 1524.30 sec
if unit in mins: 25.40 mins
if unit in hours: 0.42 hours


An average game duration of around **20 minutes** seems reasonable for a League of Legends match. The other two values—measured in seconds or hours—appear either too short or too long, further supporting the assumption that `gameDuration` is recorded in seconds.

In [21]:
avg_NaNgameDura = np.average(match_df[match_df['gameId'].isin(missingVals)]['gameDuration'].values)

print('NaN games avg: {:.2f} mins'.format(avg_NaNgameDura/60))

NaN games avg: 3.83 mins


These games are too short to provide meaningful insights for the analysis, and the missing `win` field is likely a direct result of this.

To ensure data quality, we should drop all records related to these matches.

In [22]:
winner_df = winner_df[~winner_df['gameId'].isin(missingVals)]
loser_df = loser_df[~loser_df['gameId'].isin(missingVals)]
match_df = match_df[~match_df['gameId'].isin(missingVals)]

#### ***Issue 2***

In [23]:
match_df['gameMode'].unique()

array(['CLASSIC', 'ARAM', 'ONEFORALL', 'URF', 'TUTORIAL_MODULE_3',
       'KINGPORO', 'TUTORIAL_MODULE_2'], dtype=object)

Since we are focusing only on classic games, we should drop all records from other game modes.

In [24]:
gameId_CLASSIC =  match_df[match_df['gameMode'] == 'CLASSIC'].gameId.tolist()

In [25]:
winner_df = winner_df[winner_df['gameId'].isin(gameId_CLASSIC)]
loser_df = loser_df[loser_df['gameId'].isin(gameId_CLASSIC)]
match_df = match_df[match_df['gameId'].isin(gameId_CLASSIC)]

In [26]:
match_df.gameMode.unique()

array(['CLASSIC'], dtype=object)

#### ***Issue 3***

A **remake** occurs when a player disconnects at the start, allowing their team to end the match early rather than playing a 4v5. Since this is not a true loss, it should be handled separately in our analysis.

To identify **remake** games in `match_df`:
* A team can only **remake** within the first 15 minutes.
* After 15 minutes, an attempted **remake** becomes an **early surrender**, which counts as a normal loss.
* At this ranking level, almost no matches end before 15 minutes unless a player disconnects.

Thus, we can confidently assume that matches with `gameDuration` ≤ **15 minutes** are **remakes** and should be dropped from the dataset.

In [27]:
gameId_remake = match_df.query('gameDuration <= 15*60').gameId.values.tolist()

In [28]:
winner_df = winner_df[~winner_df['gameId'].isin(gameId_remake)]
loser_df = loser_df[~loser_df['gameId'].isin(gameId_remake)]
match_df = match_df[~match_df['gameId'].isin(gameId_remake)]

In [29]:
print('Now the shortest game in df is {:.2f} mins'.format(match_df.gameDuration.min()/60))

Now the shortest game in df is 15.02 mins


## Step 3: Define the Data Model
### **3.1 Conceptual Data Model**

[View Data Model]([View Data Model](https://github.com/doshimee11/LeagueOfLegends-ETL/blob/main/DataModeling.png))

A fact-dimensional modeling approach is well-suited for this project, as the output is designed to support match-level analysis (fact), with dimensions such as champion compositions and vision control.


### **3.2 Mapping Out Data Pipelines**

The original data comes from Kaggle. After addressing data quality issues, it will be stored in an AWS RDS PostgreSQL database following these steps:

1. Create and connect to PostgreSQL.
2. Create fact and dimension tables in the database.
3. Load dimension tables first, followed by the fact table to maintain referential integrity.

## Step 4: Run Pipelines to Model the Data

### **4.1 Create the data model**
To build the data model, we need to load and transform the data efficiently.

Since the dataset is large, loading it directly into PostgreSQL is time-consuming due to network constraints. However, PostgreSQL provides greater computing power for transforming the data.

To optimize performance, we will:
1. First, load the raw data into staging tables in PostgreSQL.
2. Then, transform and move the data from staging tables to final tables for analysis.

In [ ]:
host = "host"
dbname= "dbname"
user = "user"
password = "password"
port = "port"

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(host, dbname, user, password, port))

conn.autocommit = True
cur = conn.cursor()

In [31]:
# in the meta_dfs, we only need the key-name mapping, all other columns are dropped
meta_items = meta_items[['name']]
meta_champs['key'] = meta_champs.apply(lambda row: row.data['key'], axis=1)
meta_champs = meta_champs[['key']]

In [32]:
# column name/datatype of each staging table defined, used in loading_staging_tables
match_dict = {'gameCreation': Float(),
              'gameDuration': Float(),
              'gameId': Float(),
              'gameMode': String(),
              'gameType': String(),
              'gameVersion': String(),
              'mapId': Float(),
              'participantIdentities': JSON(),
              'participants': JSON(),
              'platformId': String(),
              'queueId': Float(),
              'sessionId': Float(),
              'status.message': String(),
              'status.status_code': Float()}

winner_dict = {'teamId': Integer(),
               'win': String(),
               'firstBlood': Boolean(),
               'firstTower': Boolean(),
               'firstInhibitor': Boolean(),
               'firstBaron': Boolean(),
               'firstDragon': Boolean(),
               'firstRiftHerald': Boolean(),
               'towerKills': Integer(),
               'inhibitorKills': Integer(),
               'baronKills': Integer(),
               'dragonKills': Integer(),
               'vilemawKills': Integer(),
               'riftHeraldKills': Integer(),
               'dominionVictoryScore': Integer(),
               'bans': JSON(),
               'gameId': Float()}

loser_dict = winner_dict

In [33]:
# drop tables outlined in the 'Data Model', in case a restart is needed
def drop_tables(cur):
    games_table_drop = "DROP TABLE IF EXISTS games"
    champions_table_drop = "DROP TABLE IF EXISTS champions"
    items_table_drop = "DROP TABLE IF EXISTS items"
    objectives_visions_table_drop = "DROP TABLE IF EXISTS objectives_visions"
    champion_key_table_drop = "DROP TABLE IF EXISTS champion_key"
    item_key_table_drop = "DROP TABLE IF EXISTS item_key"
    
    # execute all queries defined
    drop_table_queries = [games_table_drop, champions_table_drop, items_table_drop, objectives_visions_table_drop, champion_key_table_drop, item_key_table_drop]
    for query in drop_table_queries:
        cur.execute(query)

In [34]:
# drop tables outlined in the 'data_modeling.pdf', in case a restart is needed
def drop_staging_tables(cur):
    staging_match_table_drop = "DROP TABLE IF EXISTS staging_match"
    staging_winner_table_drop = "DROP TABLE IF EXISTS staging_winner"
    staging_loser_table_drop = "DROP TABLE IF EXISTS staging_loser"
    staging_meta_champs_table_drop = "DROP TABLE IF EXISTS staging_meta_champs"
    staging_meta_items_table_drop = "DROP TABLE IF EXISTS staging_meta_items"
    
    # execute all queries defined
    drop_table_queries = [staging_match_table_drop, staging_winner_table_drop, staging_loser_table_drop, staging_meta_champs_table_drop, staging_meta_items_table_drop]
    for query in drop_table_queries:
        cur.execute(query)

In [35]:
# create and insert staging tables
def load_staging_tables(conn):
    
    conn = sqlalchemy.create_engine('postgresql://{}:{}@{}:{}/{}'.format(user, password, host, port, dbname))
    
    print('loading staging_match')
    match_df.to_sql('staging_match', conn, index=False, if_exists='replace', dtype=match_dict)
    
    print('loading staging_winner')
    winner_df.to_sql('staging_winner', conn, index=False, if_exists='replace', dtype=winner_dict)
    
    print('loading staging_loser')
    loser_df.to_sql('staging_loser', conn, index=False, if_exists='replace', dtype=loser_dict)
    
    print('loading staging_meta_champs')
    meta_champs.to_sql('staging_meta_champs', conn, index=True, if_exists='replace')
    
    print('creating staging_meta_items')
    meta_items.to_sql('staging_meta_items', conn, index=True, if_exists='replace')

In [36]:
# create tables outlined in the 'data_modeling.pdf'
def create_tables(cur):
    games_table_create = ("""CREATE TABLE IF NOT EXISTS games(game_id bigint PRIMARY KEY,
                                                                game_duration float NOT NULL,
                                                                game_version varchar NOT NULL,
                                                                participants varchar[10] NOT NULL)
                            """)

    champions_table_create = ("""CREATE TABLE IF NOT EXISTS champions(game_id bigint PRIMARY KEY,
                                                                        champ_1 int NOT NULL,
                                                                        champ_2 int NOT NULL,
                                                                        champ_3 int NOT NULL,
                                                                        champ_4 int NOT NULL,
                                                                        champ_5 int NOT NULL,
                                                                        champ_6 int NOT NULL,
                                                                        champ_7 int NOT NULL,
                                                                        champ_8 int NOT NULL,
                                                                        champ_9 int NOT NULL,
                                                                        champ_10 int NOT NULL)
                                """)

    items_table_create = ("""CREATE TABLE IF NOT EXISTS items(game_id bigint PRIMARY KEY,
                                                                build_1 int[6] NOT NULL,
                                                                build_2 int[6] NOT NULL,
                                                                build_3 int[6] NOT NULL,
                                                                build_4 int[6] NOT NULL,
                                                                build_5 int[6] NOT NULL,
                                                                build_6 int[6] NOT NULL,
                                                                build_7 int[6] NOT NULL,
                                                                build_8 int[6] NOT NULL,
                                                                build_9 int[6] NOT NULL,
                                                                build_10 int[6] NOT NULL)
                            """)

    objectives_visions_table_create = ("""CREATE TABLE IF NOT EXISTS objectives_visions(game_id bigint PRIMARY KEY,
                                                                                        win_dragon_soul boolean NOT NULL,
                                                                                        win_baron_nashor boolean NOT NULL,
                                                                                        win_ward_placed int NOT NULL,
                                                                                        win_ward_destroyed int NOT NULL,
                                                                                        lose_dragon_soul boolean NOT NULL,
                                                                                        lose_baron_nashor boolean NOT NULL,
                                                                                        lose_ward_placed int NOT NULL,
                                                                                        lose_ward_destroyed int NOT NULL)
                                        """)

    champion_key_table_create = ("""CREATE TABLE IF NOT EXISTS champion_key(champion_key bigint PRIMARY KEY,
                                                                            champion_name varchar NOT NULL)
                                """)
    
    item_key_table_create = ("""CREATE TABLE IF NOT EXISTS item_key(item_key bigint PRIMARY KEY,
                                                                    item_name varchar NOT NULL)
                            """)

    # execute all queries defined
    create_table_queries = [games_table_create, champions_table_create, items_table_create, objectives_visions_table_create, champion_key_table_create, item_key_table_create]
    for query in create_table_queries:
        cur.execute(query)

#### **The next cell should only be executed during the first ETL run, as it is used to recreate all tables in the database.**

In [37]:
print('drop staging tables')
drop_staging_tables(cur)

print('dropping fact/dimension tables')
drop_tables(cur)

print('creating staging tables')
load_staging_tables(cur)

print('creating fact/dimension tables')
create_tables(cur)

drop staging tables
dropping fact/dimension tables
creating staging tables
loading staging_match
loading staging_winner
loading staging_loser
loading staging_meta_champs
creating staging_meta_items
creating fact/dimension tables


In [38]:
# list all tables created
cur.execute("""SELECT table_name
              FROM information_schema.tables
              WHERE table_schema = 'public'""")

for table in cur.fetchall():
    print(table)

('staging_winner',)
('staging_loser',)
('staging_meta_champs',)
('staging_meta_items',)
('staging_match',)
('games',)
('champions',)
('items',)
('objectives_visions',)
('champion_key',)
('item_key',)


**Display all staging tables with their columns and data types**

In [39]:
pd.options.display.max_rows = 75

cur.execute("""SELECT table_name, column_name, data_type
                FROM information_schema.columns
                WHERE table_name LIKE 'staging_%'""")

pd.DataFrame(cur.fetchall(), columns=['table_name', 'column_name', 'data_type'])

,table_name,column_name,data_type
0,staging_loser,vilemawKills,integer
1,staging_winner,firstRiftHerald,boolean
2,staging_winner,towerKills,integer
3,staging_winner,inhibitorKills,integer
4,staging_winner,baronKills,integer
5,staging_winner,dragonKills,integer
6,staging_winner,vilemawKills,integer
7,staging_winner,riftHeraldKills,integer
8,staging_winner,dominionVictoryScore,integer
9,staging_winner,bans,json


#### 1. Champions Table

The **champ1 - champion 10** data is stored in the `staging_match` table, and we can retrieve the corresponding **gameId** for each of the 10 champions (**champX**).

The `champions_table_value` statement is somewhat complex due to the nature of the JSON array column in PostgreSQL.

In [40]:
champions_table_value = """
                        SELECT tb2.game_id, tb2.champ_ids[1], tb2.champ_ids[2], tb2.champ_ids[3], tb2.champ_ids[4],tb2.champ_ids[5],
                                tb2.champ_ids[6], tb2.champ_ids[7], tb2.champ_ids[8], tb2.champ_ids[9], tb2.champ_ids[10]
                        FROM
                            (SELECT tb.game_id AS game_id, array_agg(tb.c ORDER BY tb.i ASC)::jsonb[]::int[] AS champ_ids
                            FROM 
                                (SELECT "gameId" AS game_id, 
                                        json_array_elements(participants) -> 'championId' AS c, 
                                        cast(json_array_elements(participants) -> 'participantId' as jsonb)::int AS i
                                FROM staging_match) AS tb 
                            GROUP BY tb.game_id) AS tb2
                        ORDER BY tb2.game_id
                        """

In [41]:
champions_table_insert = """INSERT INTO champions(game_id, champ_1, champ_2, champ_3, champ_4, champ_5, 
                                        champ_6, champ_7, champ_8, champ_9, champ_10) {}
                        """.format(champions_table_value)

One issue that needs to be addressed before completing the insertion is that some games in the original data have missing **champ_ids**. This could potentially lead to incomplete or inconsistent data if not handled properly.

In [42]:
cur.execute(champions_table_value)
a = pd.DataFrame(cur.fetchall())
a[a.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,10
491,3.683140e+09,517,39,55,13,11,122,NaN,NaN,NaN,NaN
492,3.683149e+09,39,59,111,23,64,61,NaN,NaN,NaN,NaN
493,3.683152e+09,111,45,498,238,84,28,NaN,NaN,NaN,NaN
494,3.683205e+09,61,104,69,238,99,17,NaN,NaN,NaN,NaN
495,3.683216e+09,122,104,61,517,39,266,NaN,NaN,NaN,NaN


Upon reviewing the original `match_df`, we found that these games only have 6 participants—indicating a 1 vs 5 scenario. Such games are invalid for our analysis, as they do not reflect a proper match setup.

Therefore, these rows should be deleted from the `staging_match` table to ensure the integrity and validity of the data for our database and analysis.

In [43]:
game_with_missing_champ_ids = list(a[a.isnull().any(axis=1)][0].array)

for game in range(len(game_with_missing_champ_ids)):
    total_participants = len(match_df[match_df['gameId'].isin(game_with_missing_champ_ids)].iloc[game].participants)
    
    print('game {} has: {} participants'.format(game_with_missing_champ_ids[game], total_participants))

game 3683139583.0 has: 6 participants
game 3683149180.0 has: 6 participants
game 3683152096.0 has: 6 participants
game 3683204639.0 has: 6 participants
game 3683215743.0 has: 6 participants


In [44]:
# delete game with not 10 participants
cur.execute("""DELETE FROM staging_match
                WHERE "gameId" IN %s""", (tuple(game_with_missing_champ_ids),))

In [45]:
cur.execute(champions_table_value)
a = pd.DataFrame(cur.fetchall())
a[a.isnull().any(axis=1)]

,0,1,2,3,4,5,6,7,8,9,10


Since all games are now valid, we can proceed with the insertion of the data into the database. This will involve populating the staging tables first, followed by transforming and moving the data into the final tables for analysis.

In [46]:
cur.execute(champions_table_insert)

In [47]:
cur.execute("""SELECT * FROM champions LIMIT 3""")
cur.fetchall()

[(3190375139, 201, 13, 136, 2, 29, 497, 120, 117, 498, 121),
 (3191353856, 82, 99, 421, 38, 27, 55, 157, 497, 145, 59),
 (3191955256, 27, 117, 81, 56, 105, 145, 114, 201, 38, 72)]

#### 2. Items Table

The table can be populated with the **builds/items** information, which can be retrieved from the `staging_match` table. This table will include the items purchased by each champion during the game, providing insight into the champion builds and itemization strategies.

In [ ]:
items_table_value = """
                        SELECT game_id, i[1:1], i[2:2], i[3:3], i[4:4], i[5:5], i[6:6], i[7:7], i[8:8], i[9:9], i[10:]
                        FROM
                            (SELECT game_id AS game_id, ((array_agg(array[i0,i1,i2,i3,i4,i5,i6])))::jsonb[]::int[] AS i
                            FROM
                                (SELECT game_id AS game_id, p ->'item0' AS i0, p -> 'item1' AS i1,
                                        p ->'item2' AS i2, p ->'item3' AS i3, 
                                        p ->'item4' AS i4,p ->'item5' AS i5,p ->'item6' AS i6
                                FROM 
                                    (SELECT "gameId" AS game_id, json_array_elements(participants) -> 'stats' AS p
                                    FROM staging_match) AS tb1) AS tb2 
                            GROUP BY game_id) AS tb3
                        ORDER BY game_id
                    """

In [ ]:
items_table_insert = """INSERT INTO items(game_id, build_1, build_2, build_3, build_4, build_5, 
                                            build_6, build_7, build_8, build_9, build_10) {}
                    """.format(items_table_value)

In [50]:
cur.execute(items_table_insert)

In [51]:
cur.execute("""SELECT * FROM items LIMIT 3""")
cur.fetchall()

[(3190375139,
  [[3047, 3800, 1413, 2055, 0, 0, 3364]],
  [[1412, 2031, 3117, 3134, 0, 0, 3364]],
  [[1055, 2031, 3006, 3133, 1018, 1042, 3340]],
  [[0, 3070, 0, 3070, 3070, 3070, 0]],
  [[3057, 3111, 3044, 1042, 1036, 1036, 3340]],
  [[2421, 2031, 3096, 3117, 1028, 0, 3364]],
  [[1055, 3006, 2421, 3031, 1042, 1042, 3340]],
  [[2055, 3401, 2423, 3024, 3801, 3117, 3364]],
  [[3040, 2031, 3285, 3158, 2055, 0, 3340]],
  [[2065, 2423, 2033, 3009, 3027, 1082, 3340]]),
 (3191353856,
  [[1412, 2031, 1031, 3111, 3071, 0, 3364]],
  [[3097, 3157, 0, 3020, 3116, 1082, 3364]],
  [[2033, 3800, 2065, 1026, 3047, 1082, 3340]],
  [[2031, 1052, 3146, 3108, 3916, 3020, 3364]],
  [[1055, 1083, 3006, 3046, 3031, 0, 3340]],
  [[2055, 3050, 3092, 0, 0, 2422, 3364]],
  [[3003, 2420, 3020, 3027, 1054, 0, 3363]],
  [[3077, 3047, 1400, 1011, 1028, 3044, 3364]],
  [[3092, 2031, 3285, 3020, 2055, 0, 3364]],
  [[0, 1055, 0, 3085, 3124, 3006, 3363]]),
 (3191955256,
  [[2033, 3157, 3152, 3100, 1082, 3158, 3364]],
  

#### 3. Objective Vision Table

Objective kills can be retrieved from `staging_loser` and `staging_winner`, as these tables contain data on team-based objectives.

Vision-related data (wards placed/destroyed) can be extracted from `staging_match`, which tracks individual contributions to vision control.

In [52]:
objectives_visions_table_value = """
                                    SELECT game_id,
                                            CASE WHEN wdk >= 4 THEN TRUE ELSE FALSE END AS win_dragon_soul,
                                            CASE WHEN wbk > 0 THEN TRUE ELSE FALSE END AS win_baron_nashor,
                                            wwp AS win_ward_placed, wwk AS win_ward_killed,
                                            CASE WHEN ldk >= 4 THEN TRUE ELSE FALSE END AS lose_dragon_soul,
                                            CASE WHEN lbk > 0 THEN TRUE ELSE FALSE END AS lose_baron_nashor,
                                            lwp AS lose_ward_placed, lwk AS lose_ward_killed
                                    FROM
                                        (SELECT game_id AS game_id,
                                                avg(wdk)::int AS wdk, avg(wbk)::int AS wbk,
                                                sum(wp::jsonb::int) FILTER (WHERE win::jsonb::boolean IS TRUE) AS wwp,
                                                sum(wk::jsonb::int) FILTER (WHERE win::jsonb::boolean IS TRUE) AS wwk,
                                                avg(ldk)::int AS ldk, avg(lbk)::int AS lbk,
                                                sum(wp::jsonb::int) FILTER (WHERE win::jsonb::boolean IS FALSE) AS lwp,
                                                sum(wk::jsonb::int) FILTER (WHERE win::jsonb::boolean IS FALSE) AS lwk
                                        FROM
                                            (SELECT m."gameId" AS game_id,
                                                    json_array_elements(participants) #> '{stats, win}' AS win,
                                                    w."baronKills" AS wbk, w."dragonKills" AS wdk,
                                                    json_array_elements(participants) #> '{stats, wardsPlaced}' AS wp,
                                                    json_array_elements(participants) #> '{stats, wardsKilled}' AS wk,
                                                    l."baronKills" AS lbk, l."dragonKills" AS ldk
                                            FROM staging_match AS m  
                                            INNER JOIN staging_winner AS w ON (m."gameId" = w."gameId") 
                                            INNER JOIN staging_loser AS l ON (m."gameId" = l."gameId")
                                            ORDER BY game_id) AS tb1
                                        GROUP BY game_id) AS tb3
                                """

In [53]:
objectives_visions_table_insert = """INSERT INTO objectives_visions(game_id, win_dragon_soul, win_baron_nashor, win_ward_placed, win_ward_destroyed, 
                                                    lose_dragon_soul, lose_baron_nashor, lose_ward_placed, lose_ward_destroyed) 
{}""".format(objectives_visions_table_value)

In [54]:
cur.execute(objectives_visions_table_insert)

In [55]:
cur.execute("""SELECT * FROM objectives_visions LIMIT 3""")
cur.fetchall()

[(3190375139, False, False, 33, 5, False, False, 26, 3),
 (3191353856, False, True, 70, 23, False, False, 59, 17),
 (3191955256, False, True, 79, 21, False, False, 66, 21)]

#### 4. ChampionKey Table

Retrieved from `staging_meta_champs`, which contains metadata on all champions, including their key-name mapping used across match tables.

In [56]:
champion_key_table_value = """SELECT key::int, index
                                FROM staging_meta_champs
                                ORDER BY key::int"""

In [57]:
champion_key_table_insert = """INSERT INTO champion_key(champion_key, champion_name) {}""".format(champion_key_table_value)

In [58]:
cur.execute(champion_key_table_insert)

In [59]:
cur.execute("""SELECT * FROM champion_key LIMIT 10""")
cur.fetchall()

[(1, 'Annie'),
 (2, 'Olaf'),
 (3, 'Galio'),
 (4, 'TwistedFate'),
 (5, 'XinZhao'),
 (6, 'Urgot'),
 (7, 'Leblanc'),
 (8, 'Vladimir'),
 (9, 'Fiddlesticks'),
 (10, 'Kayle')]

#### 5. ItemKey Table

Retrieved from `staging_meta_items`, which contains the key-name mapping for all in-game items used in match data.

In [60]:
item_key_table_value = """SELECT index, name
                            FROM staging_meta_items
                            ORDER BY index"""

In [61]:
item_key_table_insert = """INSERT INTO item_key(item_key, item_name) {}""".format(item_key_table_value)

In [62]:
cur.execute(item_key_table_insert)

#### 6. Games Table

Retrieved from `staging_match`, which contains detailed match data, including champions, items, vision stats, and other in-game events.

In [63]:
games_table_value = """
                        SELECT game_id, game_duration, game_version, array_agg(a) AS participants
                        FROM
                            (SELECT "gameId" AS game_id,
                                    "gameDuration" AS game_duration,
                                    "gameVersion" AS game_version, 
                                    json_array_elements("participantIdentities") #> '{player, accountId}' AS a
                            FROM staging_match) AS tb1
                        GROUP BY game_id, game_duration, game_version
                        ORDER BY game_id
                    """

In [64]:
games_table_insert = """INSERT INTO games(game_id, game_duration, game_version, participants) {}""".format(games_table_value)

In [65]:
cur.execute(games_table_insert)

In [66]:
cur.execute("""SELECT * FROM games LIMIT 3""")
cur.fetchall()

[(3190375139,
  1021.0,
  '8.8.226.7254',
  ['"CTqmxBX1-b39i2eciDskPnWAbtu7I3cp7-HuUd12R8Cj"',
   '"Th3f97roy4k8E__GshlFHDPfP76TLE5kptO4GecDx7by"',
   '"8abTvRQwXwzT0J4tCtEP_3-mHShfeAL3EcKAkfZA7EnC"',
   '"Pd7c7I2_IbRJKfKpxyH13YmkAYXQVtX9VCmn3yjU1hCX"',
   '"ej_Yq6ot3ORIHdw9YBpBPHXtcFitnD7bShFRrWvrK54F"',
   '"qLbKO8QWzyy_ySveGsFFyTEtuRpSWU4R3MkH8SxE6iqc"',
   '"s5IirwYLAlm0bimWM6vV7eEWbJePZWWy0SywzLIqRLM"',
   '"OeD4zr0xorGoAGlFXVHVo1hf474l7JqXhKXp67AEu7B0QVc"',
   '"2j7W-hxT3CYCUSSzT3cbntQYry2bEx_ME_kybyV01KLy"',
   '"rKaQImjZTiOvZ4txjN-HVnLuhfjQlUNom9M13GwutBpi"']),
 (3191353856,
  1507.0,
  '8.9.227.7511',
  ['"h_YSz7GOD5Y5hvGOK-CW-VtdDftugqNa8Q-iCmGmCuH_"',
   '"kaXDt1S1tNADQwlVpIsFi0Caz6N8lHFXJbTQJSR_SaJxIO8"',
   '"MHQRKpJwavPjK7DKYnayXCtW_5wpbqiYO05aytyYg2IanvI"',
   '"1Re3T6aEeajR2cCNCD59CW2A8UQ4HMrA-f8RdAeJ3GolqNA"',
   '"eFcFYgwwS6YbTSRrDnILkx_Nh4usIsQjOVyU7VMoJZlF"',
   '"eB-cQ7uhwmQNh5jpjh2sdDaEx9aUd9Kx-OV15zi81YLj"',
   '"agkXHV65QfgNjgAq38Mvm6bG3Zvm74YOWjnmqr0G1HCaZAY"',

### **4.2 Data Quality Checks**
* Integrity Constraints: Ensured by **NOT NULL** and **PRIMARY KEY** constraints in the table creation queries.
* Source/Count Validation: All fact and dimension tables should have matching row counts and correctly ordered IDs to maintain data consistency.

We begin by examining the data types of each fact and dimension table to ensure consistency and correctness before proceeding with further validation.

In [67]:
# print data type of 'games'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'games' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,game_id,bigint
1,game_duration,double precision
2,game_version,character varying
3,participants,ARRAY


In [68]:
# print data type of 'champions'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'champions' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,game_id,bigint
1,champ_1,integer
2,champ_2,integer
3,champ_3,integer
4,champ_4,integer
5,champ_5,integer
6,champ_6,integer
7,champ_7,integer
8,champ_8,integer
9,champ_9,integer


In [69]:
# print data type of 'items'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'items' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,game_id,bigint
1,build_1,ARRAY
2,build_2,ARRAY
3,build_3,ARRAY
4,build_4,ARRAY
5,build_5,ARRAY
6,build_6,ARRAY
7,build_7,ARRAY
8,build_8,ARRAY
9,build_9,ARRAY


In [70]:
# print data type of 'objectives_visions'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'objectives_visions' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,game_id,bigint
1,win_dragon_soul,boolean
2,win_baron_nashor,boolean
3,win_ward_placed,integer
4,win_ward_destroyed,integer
5,lose_dragon_soul,boolean
6,lose_baron_nashor,boolean
7,lose_ward_placed,integer
8,lose_ward_destroyed,integer


In [71]:
# print data type of 'champion_key'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'champion_key' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,champion_key,bigint
1,champion_name,character varying


In [72]:
# print data type of 'item_key'
cur.execute("""SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = 'item_key' """)

pd.DataFrame(cur.fetchall(), columns=['column_name', 'data_type'])

,column_name,data_type
0,item_key,bigint
1,item_name,character varying


Verify that the fact and dimension tables have an equal number of rows to ensure data consistency and completeness across all related tables.

In [73]:
# count rows of table
count_rows = """
                SELECT
                    (SELECT count(*) FROM games) AS g,
                    (SELECT count(*) FROM champions) AS c,
                    (SELECT count(*) FROM items) AS i,
                    (SELECT count(*) FROM objectives_visions) AS o
            """

cur.execute(count_rows)
cur.fetchall()

[(87930, 87930, 87930, 87930)]

Select random samples from both the fact tables and the `staging_match` table to verify data integrity and ensure proper transformations.

In [74]:
random_games_samples = """
                            SELECT "gameDuration" AS sm_gd, gd, "gameVersion" AS sm_gv, gv
                            FROM staging_match, 
                                (SELECT g.game_id AS game_id, g.game_duration AS gd, g.game_version AS gv
                                    FROM games AS g
                                    ORDER BY random()
                                    LIMIT 3) AS tb1
                            WHERE "gameId" IN (game_id)
                        """

cur.execute(random_games_samples)
cur.fetchall()

[(1505.0, 1505.0, '10.1.303.9385', '10.1.303.9385'),
 (2453.0, 2453.0, '10.4.308.9400', '10.4.308.9400'),
 (939.0, 939.0, '10.2.305.4739', '10.2.305.4739')]

The data matches exactly.

Now, print two **random** rows from each fact and dimension table to confirm correctness and completeness.

In [75]:
# print two random rows of 'games'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'games' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM games ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(game_id,)","(game_duration,)","(game_version,)","(participants,)"
0,4226442981,1089.0,10.6.313.2784,"[""dgk5FasVGqHlMWN4H5YsN6GAJDdwf_Ofevb1qLTuBfwg..."
1,4085306032,1719.0,10.1.303.9385,"[""zaz8bj3nOMJ6NC30cvdQSnd8E9Lh8VpPSNwa09CBOJHw..."


In [76]:
# print two random rows of 'champions'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'champions' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM champions ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(game_id,)","(champ_1,)","(champ_2,)","(champ_3,)","(champ_4,)","(champ_5,)","(champ_6,)","(champ_7,)","(champ_8,)","(champ_9,)","(champ_10,)"
0,4162879427,266,60,69,497,429,81,39,11,16,79
1,4195883004,64,2,39,350,81,59,26,142,145,106


In [77]:
# print two random rows of 'items'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'items' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM items ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(game_id,)","(build_1,)","(build_2,)","(build_3,)","(build_4,)","(build_5,)","(build_6,)","(build_7,)","(build_8,)","(build_9,)","(build_10,)"
0,4237068192,"[[1055, 2421, 3053, 3812, 3071, 3047, 3340]]","[[3142, 1412, 3147, 1001, 3133, 1036, 3364]]","[[2033, 3152, 0, 3100, 3020, 3165, 3364]]","[[0, 0, 0, 0, 0, 0, 3363]]","[[3155, 3033, 3153, 3025, 3042, 3158, 3363]]","[[3860, 3151, 3191, 1082, 3020, 3108, 3364]]","[[2055, 3107, 3853, 3916, 3174, 3020, 3364]]","[[1402, 3157, 1058, 3020, 3165, 3135, 3364]]","[[2055, 3031, 3153, 3508, 3046, 3006, 3363]]","[[3124, 3042, 1055, 3101, 3108, 3006, 3363]]"
1,4186314126,"[[3373, 3387, 3047, 3024, 3075, 1011, 3363]]","[[0, 3380, 1412, 3052, 1037, 3047, 3340]]","[[3157, 3040, 3020, 1082, 1056, 3027, 3364]]","[[3389, 3140, 1036, 1055, 3085, 3006, 3363]]","[[3860, 3117, 2055, 3050, 3801, 3114, 3364]]","[[3053, 0, 3193, 3742, 3078, 3047, 3340]]","[[1055, 1036, 3006, 3031, 3508, 3072, 3363]]","[[3107, 1082, 3113, 1402, 3114, 1001, 3364]]","[[3109, 2055, 3857, 3047, 3105, 2424, 3364]]","[[2420, 3031, 3006, 3046, 3022, 1055, 3364]]"


In [78]:
# print two random rows of 'objectives_visions'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'objectives_visions' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM objectives_visions ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(game_id,)","(win_dragon_soul,)","(win_baron_nashor,)","(win_ward_placed,)","(win_ward_destroyed,)","(lose_dragon_soul,)","(lose_baron_nashor,)","(lose_ward_placed,)","(lose_ward_destroyed,)"
0,4207010740,False,True,81,28,False,False,68,30
1,4221563279,False,True,55,17,False,False,57,12


Now, verify the two key-name mapping tables to ensure the correct mapping between internal IDs and names for champions and items. These mappings should be checked for consistency and completeness.

In [79]:
# print two random rows of 'champion_key'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'champion_key' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM champion_key ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(champion_key,)","(champion_name,)"
0,25,Morgana
1,20,Nunu


In [80]:
# print two random rows of 'item_key'
cur.execute("""SELECT column_name
                FROM information_schema.columns
                WHERE table_name = 'item_key' """)

columns = cur.fetchall()

cur.execute("""SELECT * FROM item_key ORDER BY random() LIMIT 2""")
pd.DataFrame(cur.fetchall(), columns=columns)

,"(item_key,)","(item_name,)"
0,3400,'Your Cut'
1,3124,Guinsoo's Rageblade


### **4.3 Data dictionary**

#### Games - Fact Table

- ***game_id***: The unique identifier for each LOL game, loaded from `staging_match`.
- ***game_duration***: The total duration of each game, also retrieved from `staging_match`.
- ***participants***: A list of the account IDs for each of the 10 players in a game, loaded from `staging_match` → `player` → `accountId`.

#### Champions

- ***game_id***: Same as in the fact table, linking the match data.
- ***champ_1 - champ_5***: The champions from the winning team, order unspecified, retrieved from `staging_match` → `participants` → `championId`.
- ***champ_6 - champ_10***: The champions from the losing team, with the same conditions as above.

#### Items

- ***game_id***: Same as in the fact table, linking the match data.
- ***build_1 - build_5***: The builds from the winning team, each build consisting of 7 items (any item = 0 indicates fewer than 7 items). The order follows that of the champions in the `champions` table, retrieved from `staging_match` → `participants` → `item[0-6]`.
- ***build_6 - build_10***: The builds from the losing team, with the same conditions as above.

#### Objectives Visions

- ***game_id***: Same as in the fact table, linking the match data.
- ***win_drag_soul***: If the winning team killed 4 or more dragons, they have a dragon soul; otherwise, they do not.
- ***win_baron_nashor***: If the winning team killed Baron Nashor at least once, they have Baron Nashor; otherwise, they do not.
- ***win_ward_placed***: The total number of wards placed by the 5 participants in the winning team throughout the game.
- ***win_ward_killed***: The total number of wards destroyed by the 5 participants in the winning team throughout the game.
- ***lose_\****: Same conditions applied for the losing team, covering dragon soul, Baron Nashor, wards placed, and wards destroyed.

Note: All data is sourced from `staging_winner` and `staging_loser`, though some SQL functions and aggregations were applied. For readability, these specifics are not outlined here, but can be referred back to in the query `objectives_visions_value`.

#### Champion Key

- ***champion_key***: The integer key used in the `champion` table, loaded from `staging_meta_champions` → `key`.
- ***champion_name***: The text mapping for `champion_key`, loaded from `staging_meta_champions` → `index`.

#### Item Key

- ***item_key***: The integer key used in the items table, loaded from `staging_meta_items` → `index`.
- ***item_name***: The text mapping for item_key, loaded from `staging_meta_items` → `name`.

## Step 5: Project Write Up

***Rationale for the Choice of Tools and Technologies:***

Initially, AWS Redshift was considered for handling the scale of the data, but its limitations—particularly its inability to store composite keys (such as arrays) and lack of support for some advanced relational database features—made it less suitable. Given these constraints, a relational database was determined to be a more appropriate choice for storing composite JSON fields. AWS RDS with Postgres was selected as it allows for efficient handling of these complex data structures while providing flexibility in the ETL process. While Redshift may outperform Postgres in analytical queries, Postgres was favored for its ease of ETL integration.

***How Often the Data Should Be Updated and Why:***

Data should be updated at least once a week. In League of Legends, metrics can change rapidly due to frequent updates and patches. Game developers release new versions and balance changes on a weekly basis, and trends evolve quickly within the player base, making regular updates essential to ensure the data remains relevant.

***If the Data Was Increased by 100x:***

In the event of a 100x increase in data, Apache Spark and Apache Airflow should be integrated into the ETL process. Spark's distributed computing capabilities would handle the increased scale efficiently, while Airflow would provide necessary orchestration and monitoring of the ETL pipeline, especially as the processing time would increase due to the larger dataset.

***If the Data Populates a Dashboard That Must Be Updated on a Daily Basis by 7 AM Every Day:***

To meet the daily update requirement for the dashboard, the ETL process should be scheduled to run on a virtual machine, such as an AWS EC2 instance, every night. AWS CloudWatch can be used to trigger the ETL process at a set time, ensuring the dashboard is populated with the most up-to-date data by 7 AM each day.

***If the Database Needed to Be Accessed by 100+ People:***

To ensure high availability and handle the access demands of 100+ users, AWS RDS's Multi-AZ deployment should be utilized. This would provide replication across multiple availability zones for better reliability and uptime. Alternatively, using Apache Spark for a distributed database setup would ensure scalability and availability across distributed machines, allowing for efficient querying and access by multiple users simultaneously.